In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from pycaret.classification import *
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 120)

In [5]:
PATH_REPO = Path(os.getcwd()).parent
PATH_DATA = PATH_REPO / "data"
PATH_DATA_INPUT = PATH_DATA / "input"
GWG = "GWG"
TARGETS = ["diab_hiper", "LGA_SGA"]
PATH_REPO

WindowsPath('D:/git-projects/optiGWG')

In [4]:
process = pd.read_csv(PATH_DATA_INPUT / "01_process.csv", index_col=0)
with open(PATH_REPO / "data/models/numeric_features_names.pkl", 'rb') as f:
    numeric_features_names = pickle.load(f)
with open(PATH_REPO / "data/models/categoric_features_names.pkl", 'rb') as f:
    categoric_features_names = pickle.load(f)
with open(PATH_REPO / "data/models/targets_names.pkl", 'rb') as f:
    targets_names = pickle.load(f)
print("Total columns names: Numerics ({}) + Categoric ({}) + Targets ({}) = {}".format(len(numeric_features_names), len(categoric_features_names), len(targets_names), 
                                                                                       len(numeric_features_names)+ len(categoric_features_names)+ len(targets_names) ))
process

Total columns names: Numerics (12) + Categoric (89) + Targets (4) = 105


,LGA,SGA,hipertension,diabetes,edad_a,paf_no_a,paf_diab_a,paf_tbcp_a,paf_hipe_a,paf_aler_a,pam_no_a,pam_diab_a,pam_tbcp_a,pam_hipe_a,pam_aler_a,pam_asma_a,pam_epil_a,pam_tiro_a,pam_card_a,pam_rena_a,paq_no_a,paq_cono_a,paq_miom_a,paq_infe_a,paq_liti_a,paq_cole_a,paq_apen_a,mecr_no_a,mecr_dur_a,mecr_des_a,mecr_nos_a,dro_no_a,dro_coca_a,dro_mari_a,dro_past_a,dro_amfe_a,dro_benz_a,epre_nor_a,epre_mal_a,epre_pree_a,epre_diab_a,epre_snc_a,epre_car_a,epre_dow_a,epre_otr_a,ae_a,ap_a,ppr_a,pter_a,mnat_a,mneo_a,hv_a,inicontp_a,nrocontr_a,IMC,GWG,activida_a_1,activida_a_2,activida_a_3,activida_a_4,activida_a_5,activida_a_6,activida_a_infrequent_sklearn,alcohol_a_2,alcohol_a_infrequent_sklearn,anticonc_a_3,anticonc_a_4,anticonc_a_5,anticonc_a_infrequent_sklearn,cigardia_a_2,cigardia_a_3,cigardia_a_infrequent_sklearn,comuna_a_la pintana,comuna_a_pirque,comuna_a_puente alto,comuna_a_san jose maipo,comuna_a_infrequent_sklearn,est_civil_a_1,est_civil_a_2,est_civil_a_3,est_civil_a_4,estnutri_a_1,estnutri_a_2,estnutri_a_3,estnutri_a_4,estudio_a_1,estudio_a_2,estudio_a_3,estudio_a_4,estudio_a_5,grpsanc_a_1,grpsanc_a_2,grpsanc_a_3,grpsanc_a_4,grpsanc_a_5,grpsanr_a_1,grpsanr_a_2,grpsanr_a_5,grpsanr_a_infrequent_sklearn,previsio_a_1,previsio_a_2,previsio_a_3,previsio_a_4,previsio_a_5,previsio_a_6
0,0,0,0,0,39.000000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,8.000000,9.000000,38.265306,12.108655,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,24.000000,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,12.000000,6.000000,26.638918,9.000000,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,28.000000,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,0.000000,0.009408,0.008558,1.004386,12.000000,12.000000,20.719787,15.000000,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,0,0,0,0,24.000000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,0.000000,0.009408,0.008558,1.004386,23.000000,6.000000,24.141519,17.000000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,1,0,0,0,25.000000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,22.000000,6.000000,20.446742,17.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93942,0,0,0,0,27.223962,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.20584,0.002913,0.068071,0.961308,0.009408,0.008558,1.004386,12.184671,7.682786,37.443937,12.108655,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
93943,1,0,0,0,27.223962,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.20584,0.002913,0.068071,0.961308,0.009408,0.008558,1.004386,12.184671,7.682786,37.443937,12.108655,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
93944,1,0,0,0,27.223962,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.20584,0.002913,0.068071,0.961308,0.009408,0.008558,1.004386,12.184671,7.6

In [8]:
data_dict = {}
for t in TARGETS:
    data = process.copy()
    if t == "diab_hiper":
        enfs = ["hipertension", "diabetes"]
    elif t == "LGA_SGA":
        enfs = ["LGA", "SGA"]
    data[t] = data[enfs].apply(lambda x: max(x), axis=1)
    data.drop(columns=enfs, inplace=True)
    print(t)
    display(data[t].mean())
    data_dict[t] = data
    

diab_hiper


0.005247639626597976

LGA_SGA


0.4198963245234015

# Compare Models

In [9]:
exclude=['knn']
best_models_dict={}

for t in TARGETS:
    
    
    clf = setup(data = data_dict[t], target = t, train_size=0.7 ,preprocess=True,
                verbose=True, silent=True,  numeric_features=numeric_features_names+categoric_features_names, 
                feature_selection = True, remove_multicollinearity = True, feature_selection_threshold=.3, ignore_low_variance = True, fix_imbalance = True)

    best = compare_models(sort = 'AUC', exclude=exclude)
    best_models_dict[t] = {"best": best, "setup": clf, "scoring_grid": pull()}

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.6199,0.6285,0.3601,0.5798,0.4443,0.1788,0.1905,16.0590
lightgbm,Light Gradient Boosting Machine,0.6198,0.6269,0.3162,0.5926,0.4123,0.1682,0.1863,2.1240
gbc,Gradient Boosting Classifier,0.6119,0.6190,0.3070,0.5749,0.4002,0.1509,0.1672,5.9300
xgboost,Extreme Gradient Boosting,0.6126,0.6181,0.3616,0.5637,0.4405,0.1653,0.1749,5.6700
rf,Random Forest Classifier,0.6054,0.6007,0.3176,0.5566,0.4044,0.1411,0.1536,5.2450
ada,Ada Boost Classifier,0.5866,0.5871,0.3838,0.5133,0.4390,0.1225,0.1258,2.8880
et,Extra Trees Classifier,0.5867,0.5751,0.3307,0.5157,0.4030,0.1094,0.1158,7.8870
lr,Logistic Regression,0.5444,0.5724,0.5692,0.4672,0.5132,0.0928,0.0944,2.6550
lda,Linear Discriminant Analysis,0.5438,0.5722,0.5697,0.4667,0.5131,0.0919,0.0935,1.8020
qda,Quadratic Discriminant Analysis,0.5424,0.5631,0.5313,0.4631,0.4946,0.0801,0.0808,2.0270


In [14]:
for t in TARGETS:
    print("\n", t)

    plots_names = ['confusion_matrix', 'feature_all', 'auc']
    for pn in plots_names:
        path = PATH_DATA / "plots/{}/{}".format(t, pn)
        if not os.path.exists(path):
            os.makedirs(path)
 
        plot_model(best, plot = pn, save=path)
        print("file saved in", path)

    new_categoric_features_names = []
    new_numeric_features_names = []
    for f in best.feature_names_:
        for f_cat in categoric_features_names:
            if f_cat in f:
                new_categoric_features_names.append(f_cat)
        if f in numeric_features_names:
            new_numeric_features_names.append(f)


    for file_name, elem in zip(("best_model", "categoric_features_names", "numeric_features_names"),(best, new_categoric_features_names, new_numeric_features_names)):
        path = PATH_DATA / "models/{}/{}.pkl".format(t, file_name)
        
        with open(path, "wb") as of:
            pickle.dump(elem, of)
            print("file saved in", path)

    scoring_grid = pull()
    path = PATH_DATA / "output/{}".format(t)
    if not os.path.exists(path):
        os.makedirs(path)
        
    scoring_grid.to_excel(path / "scoring_grid.xlsx")
    print("file saved in", path)

    display(scoring_grid)

file saved in D:\git-projects\optiGWG\data\plots\LGA_SGA\auc
file saved in D:\git-projects\optiGWG\data\models\LGA_SGA\best_model.pkl
file saved in D:\git-projects\optiGWG\data\models\LGA_SGA\categoric_features_names.pkl
file saved in D:\git-projects\optiGWG\data\models\LGA_SGA\numeric_features_names.pkl
file saved in D:\git-projects\optiGWG\data\output\LGA_SGA


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.6199,0.6285,0.3601,0.5798,0.4443,0.1788,0.1905,16.059
lightgbm,Light Gradient Boosting Machine,0.6198,0.6269,0.3162,0.5926,0.4123,0.1682,0.1863,2.124
gbc,Gradient Boosting Classifier,0.6119,0.6190,0.3070,0.5749,0.4002,0.1509,0.1672,5.930
xgboost,Extreme Gradient Boosting,0.6126,0.6181,0.3616,0.5637,0.4405,0.1653,0.1749,5.670
rf,Random Forest Classifier,0.6054,0.6007,0.3176,0.5566,0.4044,0.1411,0.1536,5.245
ada,Ada Boost Classifier,0.5866,0.5871,0.3838,0.5133,0.4390,0.1225,0.1258,2.888
et,Extra Trees Classifier,0.5867,0.5751,0.3307,0.5157,0.4030,0.1094,0.1158,7.887
lr,Logistic Regression,0.5444,0.5724,0.5692,0.4672,0.5132,0.0928,0.0944,2.655
lda,Linear Discriminant Analysis,0.5438,0.5722,0.5697,0.4667,0.5131,0.0919,0.0935,1.802
qda,Quadratic Discriminant Analysis,0.5424,0.5631,0.5313,0.4631,0.4946,0.0801,0.0808,2.027
